In [ ]:
#Allow imports from the src directory
import sys
from pathlib import Path

project_root = Path().resolve().parents[0]
sys.path.append(str(project_root))

# Import self-made libraries to run checks
from src.black_scholes import black_scholes_price
from src.implied_vol import implied_volatility

In [4]:
#Single test on calculations

S = 100
K = 105
T = 0.5
r = 0.03
sigma_true = 0.25

opt_type = "call"

price = black_scholes_price(S, K, T, r, sigma_true, option_type=opt_type)
sigma_hat = implied_volatility(price, S, K, T, r, option_type=opt_type)

print("Single test")
print("True sigma:", sigma_true)
print("BS price:", price)
print("Recovered IV:", sigma_hat)
print("Abs error:", abs(sigma_hat - sigma_true))


Single test
True sigma: 0.25
BS price: 5.5759766440702805
Recovered IV: 0.24999999999999367
Abs error: 6.328271240363392e-15


In [5]:
#Batch test on calculations
S = 100
r = 0.03
sigma_true = 0.25

tests = [
    (80, 0.25, "call"),
    (100, 0.50, "call"),
    (120, 1.00, "call"),
    (80, 0.25, "put"),
    (100, 0.50, "put"),
    (120, 1.00, "put"),
]

print("Batch tests (calls + puts)")
for K, T, opt in tests:
    price = black_scholes_price(S, K, T, r, sigma_true, option_type=opt)
    sigma_hat = implied_volatility(price, S, K, T, r, option_type=opt)
    print(f"{opt:4} | K={K:3} | T={T:4} | price={price:9.4f} | iv={sigma_hat:.6f} | err={abs(sigma_hat - sigma_true):.2e}")


Batch tests (calls + puts)
call | K= 80 | T=0.25 | price=  20.7394 | iv=0.250000 | err=2.14e-15
call | K=100 | T= 0.5 | price=   7.7603 | iv=0.250000 | err=5.55e-17
call | K=120 | T= 1.0 | price=   4.4633 | iv=0.250000 | err=3.33e-16
put  | K= 80 | T=0.25 | price=   0.1416 | iv=0.250000 | err=5.55e-17
put  | K=100 | T= 0.5 | price=   6.2715 | iv=0.250000 | err=0.00e+00
put  | K=120 | T= 1.0 | price=  20.9168 | iv=0.250000 | err=5.00e-16


In [6]:
#Deeper stress tests with deep ITM/OTM and short maturities

S = 100
r = 0.03
sigma_true = 0.40

tests = [
    (50, 0.10, "call"),   # deep ITM call
    (150, 0.10, "call"),  # deep OTM call
    (50, 0.10, "put"),    # deep OTM put
    (150, 0.10, "put"),   # deep ITM put
    (100, 0.02, "call"),  # very short maturity ATM
    (100, 0.02, "put"),
]

print("Stress tests")
for K, T, opt in tests:
    price = black_scholes_price(S, K, T, r, sigma_true, option_type=opt)
    sigma_hat = implied_volatility(price, S, K, T, r, option_type=opt)
    print(f"{opt:4} | K={K:3} | T={T:5} | price={price:9.4f} | iv={sigma_hat} | err={abs(sigma_hat - sigma_true) if sigma_hat==sigma_hat else 'nan'}")


Stress tests
call | K= 50 | T=  0.1 | price=  50.1498 | iv=0.39999999792055246 | err=2.0794475585894645e-09
call | K=150 | T=  0.1 | price=   0.0031 | iv=0.3999999999999982 | err=1.8318679906315083e-15
put  | K= 50 | T=  0.1 | price=   0.0000 | iv=0.39999999999999947 | err=5.551115123125783e-16
put  | K=150 | T=  0.1 | price=  49.5537 | iv=0.3999999999998166 | err=1.8340884366807586e-13
call | K=100 | T= 0.02 | price=   2.2859 | iv=0.39999999999999947 | err=5.551115123125783e-16
put  | K=100 | T= 0.02 | price=   2.2259 | iv=0.3999999999999994 | err=6.106226635438361e-16
